In [5]:
import sys
import os

PROJECT_ROOT = "/Users/kevingarrison/Code Projects/Private Projects/Agents/Langchain_LECL/langchain-lecl"
SRC_PATH = os.path.join(PROJECT_ROOT, "src")

if SRC_PATH not in sys.path:
    sys.path.append(SRC_PATH)

In [ ]:
from tools.web_search_tool import WebSearchTool
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from langchain_core.prompts import (
    FewShotChatMessagePromptTemplate,
    FewShotPromptTemplate
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate, 
    ChatPromptTemplate
)

MODEL = "gpt-4o-mini"

load_dotenv()

True

In [3]:
tools = [WebSearchTool().brave_search]
llm = ChatOpenAI(temperature=0.0, model=MODEL)
creative_llm = ChatOpenAI(temperature=1.0, model=MODEL)

In [4]:
tools[0].to_json()

{'lc': 1,
 'type': 'not_implemented',
 'id': ['langchain_core', 'tools', 'structured', 'StructuredTool'],
 'repr': "StructuredTool(name='brave_search_fn', description='Search the web using Brave Search API and return results as a dict.', args_schema=<class 'langchain_core.utils.pydantic.brave_search_fn'>, func=<staticmethod(<function WebSearchTool.brave_search_fn at 0x114218720>)>)",
 'name': 'brave_search_fn'}

In [5]:
article = """What Are AI Agents and Why They Matter

AI agents are autonomous or semi-autonomous software systems designed to perform tasks on behalf of users. Unlike traditional programs that follow rigid, pre-defined steps, AI agents make decisions dynamically: they interpret goals, analyze context, and take actions to achieve outcomes.

Modern AI agents are typically powered by large language models (LLMs), giving them capabilities such as reasoning, planning, tool use, retrieval of information, and interaction with APIs or other systems. This allows them to act more like digital collaborators than simple scripts.

Key Characteristics of AI Agents

1. Autonomy
Agents can operate without continuous user input. Once given a goal, they determine the steps needed to reach it.

2. Tool Use
Agents can call APIs, run code, access databases, send emails, or control other software—bridging AI reasoning with real-world actions.

3. Memory and Adaptation
Some agent systems can store context, learn from prior tasks, and adjust their behavior over time.

4. Multi-Agent Collaboration
More advanced setups involve multiple specialized agents working together—research agents, coding agents, planning agents—similar to a small digital team.

Where AI Agents Are Used
	•	Business automation: reporting, data processing, customer support
	•	Software development: code generation, debugging, CI/CD automation
	•	Research & analysis: summarizing findings, generating insights
	•	Operations: workflow orchestration, monitoring, ticket handling
	•	Personal productivity: scheduling, email handling, task management

Why AI Agents Matter

AI agents transform the way humans interact with technology. Instead of manually navigating tools and systems, users can delegate outcomes (“Research this”, “Build that”, “Automate this workflow”). This moves work from instruction-level to goal-level, unlocking major efficiency gains.

As models improve, agents are becoming more reliable, better at planning, and safer—bringing us closer to AI that can collaborate on complex, multi-step tasks just like a capable teammate.

"""

In [6]:
system_prompt = SystemMessagePromptTemplate.from_template("You are an helpful AI Assistant for omptimizing and rewriting articles!")
human_prompt = HumanMessagePromptTemplate.from_template("Your task is to create a optimized article for the articel: {article}.", input_variables=["article"])

In [7]:
first_prompt = ChatPromptTemplate.from_messages([system_prompt, human_prompt])

In [8]:
from langchain_core.runnables import RunnablePassthrough

chain_1 = (
    {"article": lambda x: x["article"]}
    | RunnablePassthrough.assign(
        new_article=(
            first_prompt 
            | llm 
            | (lambda x: x.content)
        )
    )
)

In [9]:
chain_1.invoke({"article":article})

{'article': 'What Are AI Agents and Why They Matter\n\nAI agents are autonomous or semi-autonomous software systems designed to perform tasks on behalf of users. Unlike traditional programs that follow rigid, pre-defined steps, AI agents make decisions dynamically: they interpret goals, analyze context, and take actions to achieve outcomes.\n\nModern AI agents are typically powered by large language models (LLMs), giving them capabilities such as reasoning, planning, tool use, retrieval of information, and interaction with APIs or other systems. This allows them to act more like digital collaborators than simple scripts.\n\nKey Characteristics of AI Agents\n\n1. Autonomy\nAgents can operate without continuous user input. Once given a goal, they determine the steps needed to reach it.\n\n2. Tool Use\nAgents can call APIs, run code, access databases, send emails, or control other software—bridging AI reasoning with real-world actions.\n\n3. Memory and Adaptation\nSome agent systems can s

In [10]:
system_prompt = SystemMessagePromptTemplate.from_template("You are an helpful AI Assistant for creating summaries!")
human_prompt = HumanMessagePromptTemplate.from_template("Create summarizations of the original article: {article}", input_variables=["article",])

second_prompt = ChatPromptTemplate([system_prompt, human_prompt])
system_prompt = HumanMessagePromptTemplate.from_template("Create summarizations of the original article: {new_article}", input_variables=["new_article"])
third_prompt = ChatPromptTemplate([system_prompt, human_prompt])


In [11]:
chain_2 = (
    {"article": lambda x:x['article']}
    | {"new_article": lambda x:x["new_article"]}
    | RunnablePassthrough.assign(
    summary_original=(
    second_prompt
    | llm
    | {"summary_original": lambda x: x.content}),
    summary_optimized=(
    third_prompt
    | llm
    | {"summary_optimized": lambda x: x.content})
    )
)


In [12]:
chain_3 = chain_1 | chain_2

In [13]:
result = chain_3.invoke({"article":article})

In [14]:
print(result["article"])
print("------")
print(result["new_article"])
print("------")
print(result["summary_original"])
print("------")
print(result["summary_optimized"])

What Are AI Agents and Why They Matter

AI agents are autonomous or semi-autonomous software systems designed to perform tasks on behalf of users. Unlike traditional programs that follow rigid, pre-defined steps, AI agents make decisions dynamically: they interpret goals, analyze context, and take actions to achieve outcomes.

Modern AI agents are typically powered by large language models (LLMs), giving them capabilities such as reasoning, planning, tool use, retrieval of information, and interaction with APIs or other systems. This allows them to act more like digital collaborators than simple scripts.

Key Characteristics of AI Agents

1. Autonomy
Agents can operate without continuous user input. Once given a goal, they determine the steps needed to reach it.

2. Tool Use
Agents can call APIs, run code, access databases, send emails, or control other software—bridging AI reasoning with real-world actions.

3. Memory and Adaptation
Some agent systems can store context, learn from pri

In [15]:
from pydantic import BaseModel, Field

class Paragraph(BaseModel):
    original_paragraph: str = Field(description="The original pragraph")
    edited_paragraph: str = Field(description="The original pragraph")
    feedback_original: int = Field(description="Constructive output on the original paragraph")
    feedback_edited: int = Field(description="Constructive output on the otimized paragraph")

structured_llm = creative_llm.with_structured_output(Paragraph)

system_prompt = SystemMessagePromptTemplate.from_template("You are an helpful AI Assistant for giving feedback")
human_prompt = HumanMessagePromptTemplate.from_template("Create feedback of the original article summary: {summary_original}, and the optimized article summary: {summary_optimized}", input_variables=["summary_original", "summary_optimized"])

fourth_prompt = ChatPromptTemplate([system_prompt, human_prompt])

chain_4 = (
    {
     "summary_original": lambda x: x["summary_original"],
     "summary_optimized": lambda x:x["new_article"],
    }
    | fourth_prompt
    | structured_llm
)

final_chain = chain_3 | chain_4

In [16]:
final_result = final_chain.invoke({"article":article})

In [21]:
from langchain_core.prompts import AIMessagePromptTemplate

human = HumanMessagePromptTemplate.from_template(template="{input}", input_variables=["input"])
ai = AIMessagePromptTemplate.from_template(template="{output}", input_variables=["output"])

example_promt = ChatPromptTemplate.from_messages([
    human,
    ai
])

In [7]:
examples = [
    {
        "input": "A small feline with whiskers, sharp claws, and a playful nature. Often kept as a house pet.",
        "output": "Cat"
    },
    {
        "input": "A tall mammal with a long neck used to reach high leaves in trees. Found in African savannas.",
        "output": "Giraffe"
    },
    {
        "input": "A cold-blooded reptile with scales and the ability to shed its skin. Commonly long with no legs.",
        "output": "Snake"
    },
    {
        "input": "A large marine mammal that breathes air through a blowhole and uses echolocation.",
        "output": "Dolphin"
    },
    {
        "input": "A flying insect with black and yellow stripes that produces honey.",
        "output": "Bee"
    }
]

In [ ]:
from langchain_core.prompts import FewShotChatMessagePromptTemplate, 

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_promt,
    examples=examples)

print(few_shot_prompt.format())

Human: A small feline with whiskers, sharp claws, and a playful nature. Often kept as a house pet.
AI: Cat
Human: A tall mammal with a long neck used to reach high leaves in trees. Found in African savannas.
AI: Giraffe
Human: A cold-blooded reptile with scales and the ability to shed its skin. Commonly long with no legs.
AI: Snake
Human: A large marine mammal that breathes air through a blowhole and uses echolocation.
AI: Dolphin
Human: A flying insect with black and yellow stripes that produces honey.
AI: Bee


In [24]:
system_prompt = SystemMessagePromptTemplate.from_template(template="You are a AI assistant you classify species based on descriptions.")
user_prompt = HumanMessagePromptTemplate.from_template(template="The animal can fly and has a red body with black dots.")
prompt_template = ChatPromptTemplate.from_messages([
    system_prompt,
    user_prompt
])
few_shot_prompt_template = ChatPromptTemplate.from_messages([
    system_prompt,
    few_shot_prompt,
    user_prompt
])


normal_chain = prompt_template | llm

few_shot_chain = few_shot_prompt_template | llm

normal_result = normal_chain.invoke({})
few_shot_result = few_shot_chain.invoke({})

In [27]:
print(normal_result.content)

Based on your description, the animal you are referring to is likely a ladybug, specifically a species of ladybird beetle. Ladybugs are known for their distinctive red bodies with black spots and are capable of flight. If you have more specific details or need further classification, feel free to provide additional information!


In [28]:
print(few_shot_result.content)

Ladybug (or Ladybird)


In [31]:
from langchain_core.prompts import MessagesPlaceholder

system_prompt = "You are a helpful assistant called zeta."

prompt_template = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(system_prompt),
    MessagesPlaceholder(variable_name="chat_history"),
    HumanMessagePromptTemplate.from_template("{query}")
])

In [35]:
prompt_template.format(query="Test",chat_history=[])

'System: You are a helpful assistant called zeta.\nHuman: Test'

In [36]:
pipe = prompt_template | llm

In [37]:
from langchain_core.chat_history import InMemoryChatMessageHistory

chat_map = {}

def get_chat_history(session_id: str) -> InMemoryChatMessageHistory:
    if session_id not in chat_map:
        chat_map[session_id] = InMemoryChatMessageHistory()
    return chat_map[session_id]

In [40]:
from langchain_core.runnables.history import RunnableWithMessageHistory

pipeline_with_history = RunnableWithMessageHistory(
    pipe,
    get_session_history=get_chat_history,
    input_messages_key="query",
    history_messages_key="chat_history"
)

In [41]:
result_with_memory = pipeline_with_history.invoke({"query":"Hi, my name is James"},
                             config={"session_id": "id_123"}
                             )

In [43]:
result_with_memory_1 = pipeline_with_history.invoke({"query":"What is my name"},
                             config={"session_id": "id_123"}
                             )

In [ ]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.messages import BaseMessage

class BufferWindowMessageHistory(BaseChatMessageHistory, BaseModel):
    messages: list[BaseMessage] = Field(default_factory=list)
    k: int = Field(default_factory=int)

    def __init__(self, k:int):
        super().__init__(k=k)
        print()

In [ ]:
from langchain_classic.agents import AgentExecutor

In [46]:
from IPython.display import display, Markdown

display(Markdown(result_with_memory_1.content))

Your name is James. How can I help you today, James?

In [ ]:
from langchain.m
from langchain_core.runnables import RunnableSerializable
from langchain.tools import tool

@tool
def add(x:float, y:float)->float:
    """function to add two decimal numbers"""
    return x + y

@tool
def final_answer(answer: str)->str:
    """function to create the final answer"""
    return {"final_answer": answer}



system_message = SystemMessagePromptTemplate.from_template("You are a helpful ai assistant. When answering a users question you should use a tool provided."
                                                            "After using a tool  the answer will be provided in the scratchpad below. "
                                                            "If you have an answer in the scratchpad you should answer!")
human_message = HumanMessagePromptTemplate.from_template("{input}")
prompt = ChatPromptTemplate.from_messages([
    system_message,
    human_message
])

tools = [add, final_answer]

agent : RunnableSerializable = (
    {
     "input": lambda x:x["input"],
     "chat_history":lambda x:x["chat_history"],
     "agent_scratchpad": lambda x:x.get("agent_scratchpad", [])
     }
     | prompt
     | llm.bind_tools(tools, tool_choice="any")
)

tool_call = agent.invoke({"input":"what is 5 + 5", "chat_history": []})


NameError: name 'SystemMessagePromptTemplate' is not defined

In [3]:
add

StructuredTool(name='add', description='function to add two decimal numbers', args_schema=<class 'langchain_core.utils.pydantic.add'>, func=<function add at 0x111e5c220>)

In [98]:
tool_call

AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 122, 'total_tokens': 139, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CfAUlLFTZQXudgbAggKX3GBtEeNng', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--e3de1f7d-ed58-40f7-8841-54a1959742a8-0', tool_calls=[{'name': 'add', 'args': {'x': 5, 'y': 5}, 'id': 'call_KVNlH7ttY9BCvH6tVqtfj6lu', 'type': 'tool_call'}], usage_metadata={'input_tokens': 122, 'output_tokens': 17, 'total_tokens': 139, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [99]:
name2tool = {tool.name: tool.func for tool in tools}

tool_exec_content = name2tool[tool_call.tool_calls[0]["name"]](**tool_call.tool_calls[0]["args"])

In [100]:
tool_exec_content

10

In [103]:
from langchain_core.messages import ToolMessage

tool_exec = ToolMessage(content=f"The tool returned {tool_exec_content}", tool_call_id=tool_call.tool_calls[0]["id"])

In [104]:
tool_exec

ToolMessage(content='The tool returned 10', tool_call_id='call_KVNlH7ttY9BCvH6tVqtfj6lu')

In [105]:
tool_call

AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 122, 'total_tokens': 139, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CfAUlLFTZQXudgbAggKX3GBtEeNng', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--e3de1f7d-ed58-40f7-8841-54a1959742a8-0', tool_calls=[{'name': 'add', 'args': {'x': 5, 'y': 5}, 'id': 'call_KVNlH7ttY9BCvH6tVqtfj6lu', 'type': 'tool_call'}], usage_metadata={'input_tokens': 122, 'output_tokens': 17, 'total_tokens': 139, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [106]:
result = agent.invoke({
    "input": lambda x:x["input"],
    "chat_history": [],
    "agent_scratchpad":[tool_call, tool_exec]})
result

AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 37, 'prompt_tokens': 130, 'total_tokens': 167, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CfAVwP3mtupWvBuZEHXQOJ9RMww3U', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--32ad7e02-5b69-4b40-8005-a5fb806ba973-0', tool_calls=[{'name': 'final_answer', 'args': {'answer': "It seems like you've entered a function reference or a code snippet. Could you please clarify your question or provide more context?"}, 'id': 'call_0aNb0gKPw1Kl24rLgPWInm40', 'type': 'tool_call'}], usage_metadata={'input_tokens': 130, 'output_tokens': 37, 'total_tokens': 167, 'input_token_

In [ ]:
from tools.web_search_tool import WebSearchTool
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from langchain_core.prompts import (
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    AIMessagePromptTemplate, 
    ChatPromptTemplate
    FewShotChatMessagePromptTemplate,
    FewShotPromptTemplate,
)

In [10]:
human = HumanMessagePromptTemplate.from_template(template="{input}", input_variables=["input"])
ai = AIMessagePromptTemplate.from_template(template="{output}", input_variables=["output"])

example_promt = ChatPromptTemplate.from_messages([
    human,
    ai
])

NameError: name 'AIMessagePromptTemplate' is not defined

In [ ]:
dir(FewShotPromptTemplate(input_variables=[], example_promt=example_promt))

ValidationError: 1 validation error for FewShotPromptTemplate
  Value error, One of 'examples' and 'example_selector' should be provided [type=value_error, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.12/v/value_error